Importing the required modules

In [ ]:
import cv2 as cv
import dlib
import imageio
import numpy as np


Reading the two images that will be used for image morphing

In [ ]:
img1 = cv.imread("images/image1.jpg")
img1 = cv.cvtColor(img1, cv.COLOR_BGR2RGB)

img2 = cv.imread("images/image2.jpg")
img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)


Create empty lists to store the tie points

In [ ]:
points1 = []
points2 = []


Defining functions that will be used later in the program

Function that reads tie points from an input file

In [ ]:
def tiePointsf():
    # Create empty lists to store the tie points
    points1 = []
    points2 = []

    # Open the input file and read the number of tie points
    with open("input.txt") as f:
        for _ in range(int(f.readline())):
            # Read each line of tie point coordinates and add them to their respective lists
            x1, y1, x2, y2 = [int(i) for i in f.readline().split()]

            points1.append((x1, y1))
            points2.append((x2, y2))

    # Add the corner points to the list of points for the first image
    points1.append((0, 0))
    points1.append((img1.shape[1] - 1, 0))
    points1.append((0, img1.shape[0] - 1))
    points1.append((img1.shape[1] - 1, img1.shape[0] - 1))

    # Add the corner points to the list of points for the second image
    points2.append((0, 0))
    points2.append((img2.shape[1] - 1, 0))
    points2.append((0, img2.shape[0] - 1))
    points2.append((img2.shape[1] - 1, img2.shape[0] - 1))

    return points1, points2


Function to get tie points using pre-trained shape predictor data

In [ ]:
# Creating a dlib face detector object for detecting faces in images
detector = dlib.get_frontal_face_detector()

# Creating a dlib shape predictor object for predicting facial landmarks
# The "shape_predictor_68_face_landmarks.dat" file contains pre-trained shape predictor data for detecting 68 facial landmarks
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


def tiePoints(img):
    # Use the detector object to detect faces in the input image
    face = detector(img)[0]

    # Use the predictor object to get the 68 facial landmark points for the detected face
    points = [(i.x, i.y) for i in predictor(img, face).parts()]

    # Add the corner points of the image to the list of landmark points
    points.append((0, 0))
    points.append((img.shape[1] - 1, 0))
    points.append((0, img.shape[0] - 1))
    points.append((img.shape[1] - 1, img.shape[0] - 1))

    # Return the list of tie points
    return points


Performs delaunay triangulation on a list of landmark points on an image and returns a list of triangle indices

In [ ]:
def delaunay_triangulation():
    # Create a rectangle that bounds the image
    bounding_rect = (0, 0, img1.shape[1], img1.shape[0])

    # Create a new Subdiv2D object using the bounding rectangle
    subdiv = cv.Subdiv2D(bounding_rect)

    # Insert the landmark points into the Subdiv2D object
    subdiv.insert(points1)

    # Get a list of triangles from the Subdiv2D object
    triangle_list = subdiv.getTriangleList()

    # Create a dictionary that maps each landmark point to its index in the points list
    point_index_dict = {point: index for index, point in enumerate(points1)}

    # Create a list of triangles where each triangle is represented as a list of three point indices
    triangle_indices = []
    for triangle in triangle_list:
        # Extract the indices of the three points that make up the triangle
        point_indices = (
            point_index_dict[(triangle[0], triangle[1])],
            point_index_dict[(triangle[2], triangle[3])],
            point_index_dict[(triangle[4], triangle[5])],
        )
        triangle_indices.append(point_indices)

    # Return the list of triangle indices
    return triangle_indices


Calculates the morphed triangle corresponding to triangle1 and triangle2 in the morphed image at the given alpha value

In [ ]:
# Resulting image after morphing
result = np.zeros_like(img1)


def compute_morphed_triangle(triangle1, triangle2, triangle, alpha):
    """
    triangle1: Contains the (x, y) coordinates of the vertices of triangle1 in img1
    triangle2: Contains the (x, y) coordinates of the vertices of triangle2 in img2
    triangle: Contains the (x, y) coordinates of the vertices of the morphed triangle in the morphed image
    """
    # Calculate the bounding rectangle for triangle1 and crop the triangle
    rect1 = cv.boundingRect(triangle1)
    x1, y1, w1, h1 = rect1
    cropped_triangle1 = img1[y1 : y1 + h1, x1 : x1 + w1]

    # Offset the points in the triangle so they are relative to the bounding rectangle
    t1_offset = triangle1 - [x1, y1]

    # Create a mask for the cropped triangle1
    mask1 = np.zeros((h1, w1), np.uint8)
    cv.fillConvexPoly(mask1, t1_offset, (1.0, 1.0, 1.0), 16, 0)

    # Apply the mask to the cropped triangle1
    cropped_triangle1 = cv.bitwise_and(cropped_triangle1, cropped_triangle1, mask=mask1)

    # Calculate the bounding rectangle for triangle2 and crop the triangle
    rect2 = cv.boundingRect(triangle2)
    x2, y2, w2, h2 = rect2
    cropped_triangle2 = img2[y2 : y2 + h2, x2 : x2 + w2]

    # Offset the points in the triangle so they are relative to the bounding rectangle
    t2_offset = triangle2 - [x2, y2]

    # Create a mask for the cropped triangle2
    mask2 = np.zeros((h2, w2), np.uint8)
    cv.fillConvexPoly(mask2, t2_offset, (1.0, 1.0, 1.0), 16, 0)

    # Apply the mask to the cropped triangle2
    cropped_triangle2 = cv.bitwise_and(cropped_triangle2, cropped_triangle2, mask=mask2)

    # Calculate the bounding rectangle for the morphed triangle and create a mask for it
    r = cv.boundingRect(triangle)
    x, y, w, h = r

    # Offset the points in the triangle so they are relative to the bounding rectangle
    t_offset = triangle - [x, y]

    # Create a mask for the morphed_triangle
    mask = np.zeros((h, w, 3), np.uint8)
    cv.fillConvexPoly(mask, t_offset, (1.0, 1.0, 1.0), 16, 0)

    # Transform triangle1 and triangle2 to the morphed triangle
    t1_offset = np.float32(t1_offset)
    t2_offset = np.float32(t2_offset)
    t_offset = np.float32(t_offset)

    # Calculate the affine transformation matrices for the first and second triangles
    M1 = cv.getAffineTransform(t1_offset, t_offset)
    wrap_triangle1 = cv.warpAffine(cropped_triangle1, M1, (w, h))

    M2 = cv.getAffineTransform(t2_offset, t_offset)
    wrap_triangle2 = cv.warpAffine(cropped_triangle2, M2, (w, h))

    # Combine the two triangles based on the alpha value to get the morphed triangle
    morphed_triangle = (1.0 - alpha) * wrap_triangle1 + alpha * wrap_triangle2
    morphed_triangle = np.uint8(morphed_triangle)

    # Add the morphed triangle to the new image using the mask
    result[y : y + h, x : x + w] = ((1.0, 1.0, 1.0) - mask) * result[
        y : y + h, x : x + w
    ] + morphed_triangle * mask


Function to create gif of image morphing

In [ ]:
def create_morph_gif(filename):
    global result

    # Create a list of triangle indices
    triangle_indices = delaunay_triangulation()

    # Initialize empty list to store frames of the gif
    frames = []

    for alpha in np.linspace(0, 1, 100):
        # Calculate intermediate points using weighted average
        morphed_points = []

        for i in range(len(points1)):
            x = (1 - alpha) * points1[i][0] + alpha * points2[i][0]
            y = (1 - alpha) * points1[i][1] + alpha * points2[i][1]
            morphed_points.append((x, y))

        for t in triangle_indices:
            # Get the corresponding triangles in both images and morphed image
            triangle1 = np.array(
                [points1[t[0]], points1[t[1]], points1[t[2]]], np.int32
            )
            triangle2 = np.array(
                [points2[t[0]], points2[t[1]], points2[t[2]]], np.int32
            )
            triangle = np.array(
                [morphed_points[t[0]], morphed_points[t[1]], morphed_points[t[2]]],
                np.int32,
            )

            # Calculate the morphed triangle and add to result
            compute_morphed_triangle(triangle1, triangle2, triangle, alpha)

        # Apply median filtering to the result and add it to the frames list
        frames.append(cv.medianBlur(result, 5))

        # Reset result for next iteration
        result = np.zeros_like(img1)

    # Save frames as a gif
    imageio.mimsave(filename, frames, fps=50)


Part A

In [ ]:
# Initialize the lists to store the tie points
points1, points2 = tiePointsf()

# Call create_morph_gif to get output1.gif
create_morph_gif("output1.gif")


Part B

In [ ]:
# Initialize the lists to store the tie points
points1 = tiePoints(img1)
points2 = tiePoints(img2)

# Call create_morph_gif to get output2.gif
create_morph_gif("output2.gif")
